---
author: [Javier Bejarano Jimenez, Finley Banas, Joseph Oforkansi, Anupraj Guragain, Anton Hedlund]
title: What is the performance difference, measured in time when running a doubling experiment between a SLL queue, a DLL queue and a queue built with an array based list when performing basic operations?
page-layout: full
categories: [post, queues, linked lists, doubling experiment]
date: "2025-03-28"
date-format: long
toc: true
format:
  html:
    code-links:    
      - text: Github Repository
        icon: github
        href: https://github.com/josephoforkansi/Algorithm-Analysis-All-Hands-Project-Module-2
    code-fold: true
    code-summary: "Show the code"
---


# Introduction

Efficient data structures are crucial for software performance, scalability, and responsiveness. Among these, queues are fundamental, supporting applications such as task scheduling, messaging systems, and real-time data processing. This project investigates the performance differences between three queue implementations: Singly Linked List (SLL), Doubly Linked List (DLL), and Array-based Queue. 

Our research question is: **What are the performance differences between SLL queue, DLL queue, and Array-based queue implementations when performing basic operations (e.g., `addfirst`, `addlast`, `removefirst`, `removelast`, `add (+)`, and `iadd (+=)`)?** 

We conducted benchmarking experiments using `SystemSense` to analyze these implementations. Key aspects considered include:

- **Algorithmic Complexity**: Evaluating time and space complexity to identify trade-offs.
- **Concurrency Considerations**: Assessing behavior in multi-threaded environments.
- **Use Case Optimization**: Identifying scenarios where each implementation excels.
- **Benchmarking Methodology**: Designing experiments to measure execution times and scaling behavior.

This project aims to provide insights into the efficiency of these queue implementations and guide the selection of an optimal data structure based on application requirements.

---

## Motivation

Efficient data structures are critical in real-world applications such as scheduling systems, task management, and networking. Different queue implementations offer trade-offs in performance. This project benchmarks these trade-offs to analyze execution times for various queue operations.

---

# Queue Implementations Analysis

## Queue Structure and FIFO Principle

Queues adhere to the **First-In-First-Out (FIFO)** principle, where elements are added at the rear and removed from the front, ensuring sequential processing.

## Implementations Overview

This project explores three queue implementations:

1. **Singly Linked List (SLL) Queue**:
   - Uses one-directional nodes with `next` references.
   - Maintains both head and tail pointers for efficient operations.
   - Each node stores only the value and a `next` reference.

2. **Doubly Linked List (DLL) Queue**:
   - Uses bidirectional nodes with both `prev` and `next` references.
   - Maintains both head and tail pointers.
   - Each node stores value, `prev`, and `next` references.

3. **Array-based Queue**:
   - No explicit node structure; uses a container of elements.
   - Optimized for operations at both ends.

---

## Key Operations

All implementations support the following core operations:

- **`enqueue`**: Add an element to the rear.
- **`dequeue`**: Remove an element from the front.
- **`peek`**: View the front element without removing it.
- **`__add__`**: Concatenate two queues.
- **`__iadd__`**: In-place concatenation. 

### Example Implementations

#### Enqueue Operation (SLL)


In [ ]:
def enqueue(self, value: Any) -> None:
    """Add an element to the end of the queue. O(1) operation using tail pointer."""
    new_node = Node(value)
    if self.is_empty():
        self.head = new_node
    else:
        self.tail.next = new_node  # Directly append at tail
    self.tail = new_node  # Update tail pointer
    self.size += 1

#### Dequeue Operation (DLL)

```python
def dequeue(self) -> Any:
    """Remove and return the first element from the queue. O(1) operation."""
    if self.is_empty():
        raise IndexError("Queue is empty")
    value = self.head.value
    self.head = self.head.next
    if self.head is None:
        self.tail = None
    else:
        self.head.prev = None
    self.size -= 1
    return value
```

#### Queue Concatenation (Array-based)

```python
def __add__(self, other: Any) -> Any:
    """Concatenate two queues. O(n) operation."""
    result = ListQueueDisplay()
    result.items = deque(self.items)  # Copy first queue
    result.items.extend(other.items)  # Append second queue
    return result
```

#### Removelast Operation (Array-based)

```python
def removelast(self) -> Any:
    """Remove and return the last element from the queue. O(1) operation."""
    if self.is_empty():
        raise IndexError("Queue is empty")
    return self.items.pop()  # O(1) operation for deque
```

#### Timing Mechanism

```python
def time_operation(func):
    """Time an operation using high-precision counter."""
    try:
        # Warm up
        func()

        # Actual timing
        start_time = perf_counter()
        func()
        elapsed = perf_counter() - start_time
        return elapsed
    except Exception as e:
        console.print(f"[red]Error during operation: {str(e)}[/red]")
        return float("nan")
```

#### Doubling Experiment

```python
def doubling(
    initial_size: int = typer.Option(10000, help="Initial size for doubling experiment"),
    max_size: int = typer.Option(1000000, help="Maximum size for doubling experiment"),
    dll: bool = typer.Option(True, help="Test DLL implementation"),
    sll: bool = typer.Option(True, help="Test SLL implementation"),
    array: bool = typer.Option(True, help="Test Array implementation"),
):
    """Run doubling experiment on queue implementations."""
    # Create results directory if it doesn't exist
    results_dir = Path("results")
    results_dir.mkdir(exist_ok=True)

    sizes = []
    current_size = initial_size
    while current_size <= max_size:
        sizes.append(current_size)
        current_size *= 2

    # Dictionary to store all results for plotting
    all_results = {}

    for approach, queue_class in QUEUE_IMPLEMENTATIONS.items():
        if not (
            (approach == QueueApproach.dll and dll)
            or (approach == QueueApproach.sll and sll)
            or (approach == QueueApproach.array and array)
        ):
            continue

        try:
            console.print(f"\n{approach.value.upper()} Queue Implementation")
            results = {
                "enqueue": [],
                "dequeue": [],
                "peek": [],
                "concat": [],
                "iconcat": [],
                "removelast": [],
            }

            for size in sizes:
                queue = queue_class()
                other = queue_class()

                # Enqueue
                enqueue_time = time_operation(
                    lambda: [queue.enqueue(i) for i in range(size)]
                )
                results["enqueue"].append(enqueue_time)

                # Dequeue
                dequeue_time = time_operation(
                    lambda: [queue.dequeue() for _ in range(size // 2)]
                )
                results["dequeue"].append(dequeue_time)

                # Refill queue
                for i in range(size // 2):
                    queue.enqueue(i)

                # Peek
                peek_time = time_operation(
                    lambda: [queue.peek() for _ in range(size // 3)]
                )
                results["peek"].append(peek_time)

                # Prepare other queue for concat
                for i in range(size // 10):
                    other.enqueue(i)

                # Concat
                concat_time = time_operation(lambda: queue + other)
                results["concat"].append(concat_time)

                # Iconcat
                iconcat_time = time_operation(lambda: queue.__iadd__(other))
                results["iconcat"].append(iconcat_time)

                # Removelast - test with fixed number of operations (100)
                removelast_time = time_operation(
                    lambda: [queue.removelast() for _ in range(100)]
                )
                results["removelast"].append(removelast_time)

            # Store results for plotting
            all_results[approach.value] = results

            # Display results in table
            table = Table(
                title=f"{approach.value.upper()} Queue Doubling Experiment Results",
                box=box.ROUNDED,
                show_header=True,
                header_style="bold magenta",
                width=250
            )
            table.add_column("Size (n)", justify="right", width=12)
            table.add_column("enq (ms)", justify="right", width=15)
            table.add_column("deq (ms)", justify="right", width=15)
            table.add_column("peek (ms)", justify="right", width=15)
            table.add_column("cat (ms)", justify="right", width=15)
            table.add_column("icat (ms)", justify="right", width=15)
            table.add_column("rml (ms)", justify="right", width=15)

            for i, size in enumerate(sizes):
                row = [f"{size:,}"]
                for operation in results.keys():
                    value = results[operation][i]
                    if np.isnan(value):  # Check for NaN
                        row.append("N/A")
                    else:
                        row.append(f"{value * 1000:.5f}")  # Show 5 decimal places
                table.add_row(*row)

            console.print(Panel(table))

        except Exception as e:
            console.print(f"[red]Error testing {approach.value}: {str(e)}[/red]")
            import traceback
            console.print(traceback.format_exc())

    # Generate and save plots
    plot_results(sizes, all_results, results_dir)
    console.print(f"[green]Plots saved to [bold]{results_dir}[/bold] directory[/green]")
```

---

## Running and Using the Tool

The benchmarking supports three queue implementations:
- DLL (Doubly Linked List)
- SLL (Singly Linked List)
- Array-based Queue

### Setting Up

To run the benchmarking tool, ensure you have Poetry installed onto your device. Navigate to the project directory and install dependencies if you have not already:

`cd analyze && poetry install`

### Running the Experiments

The tool provides two main benchmarking experiments which can also be access by 

`poetry run analyze --help`

#### Doubling Experiment

To run the doubling experiment, execute:

`poetry run analyze doubling`

This experiment measures how performance will scale with the increasing input sizes. 

You can also run:
`poetry run analyze doubling --help`
for more details and detailed apporach

#### Implementation Performance Analysis

To analyze the performance of individual queue operations, run:

`poetry run analyze analyze`

This command will provide execution times for operations like `peek`, `dequeue`, and `enqueue` to compare their efficiency.

You can also run:
`poetry run analyze analyze --help`
for more details and detailed apporach

## Output Analysis

#### Run of Doubling Experiment

##### MacOS

- Run of `systemsense`

```cmd
Displaying System Information

╭───────────────────────────────────────────────────────── System Information ─────────────────────────────────────────────────────────╮
│ ╭──────────────────┬────────────────────────────────────────────────────────────────────────────────────────╮                        │
│ │ System Parameter │ Parameter Value                                                                        │                        │
│ ├──────────────────┼────────────────────────────────────────────────────────────────────────────────────────┤                        │
│ │ battery          │ 73.00% battery life remaining, 7:20:00 seconds remaining                               │                        │
│ │ cpu              │ arm                                                                                    │                        │
│ │ cpucores         │ 11 cores                                                                               │                        │
│ │ cpufrequencies   │ Min: Unknown Mhz, Max: Unknown Mhz                                                     │                        │
│ │ datetime         │ 2025-04-28 21:09:46.967008                                                             │                        │
│ │ disk             │ Using 14.74 GB of 460.43 GB                                                            │                        │
│ │ hostname         │ MacBook-Pro-Anton.local                                                                │                        │
│ │ memory           │ Using 7.55 GB of 18.00 GB                                                              │                        │
│ │ platform         │ macOS-15.3.2-arm64-arm-64bit                                                           │                        │
│ │ pythonversion    │ 3.12.8                                                                                 │                        │
│ │ runningprocesses │ 669 running processes                                                                  │                        │
│ │ swap             │ Using 1.10 GB of 2.00 GB                                                               │                        │
│ │ system           │ Darwin                                                                                 │                        │
│ │ systemload       │ Average Load: 3.11, CPU Utilization: 29.70%                                            │                        │
│ │ virtualenv       │ /Users/antonhedlund/Library/Caches/pypoetry/virtualenvs/queue-analysis-2LJggUpT-py3.12 │                        │
│ ╰──────────────────┴────────────────────────────────────────────────────────────────────────────────────────╯                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Displaying Benchmark Results

╭───────────────────────────────────────────────────────── Benchmark Results ──────────────────────────────────────────────────────────╮
│ ╭────────────────┬───────────────────────────────────────────────────────────────╮                                                   │
│ │ Benchmark Name │ Benchmark Results (sec)                                       │                                                   │
│ ├────────────────┼───────────────────────────────────────────────────────────────┤                                                   │
│ │ addition       │ [0.315758167009335, 0.3145883330143988, 0.31581891601672396]  │                                                   │
│ │ concatenation  │ [1.7665895420359448, 1.76266020903131, 1.7622904580202885]    │                                                   │
│ │ exponentiation │ [2.23918766702991, 2.237772374995984, 2.2365284170373343]     │                                                   │
│ │ multiplication │ [0.3268889999599196, 0.3260872920509428, 0.324562625028193]   │                                                   │
│ │ rangelist      │ [0.08542008401127532, 0.0833578750025481, 0.0837147919810377] │                                                   │
│ ╰────────────────┴───────────────────────────────────────────────────────────────╯                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
```

##### Windows

#### Run of Performance Analysis

#### Summary of the results

1. **`Array Queue`** is the best for `enqueue` and `dequeue` operations.  
   `Enqueue` adds an element to the back of the queue, and `dequeue` removes an element from the front of the queue.

   `Array Queue` is the fastest → ~4.5x faster than `SLL` and ~6x faster than `DLL`. The `Array Queue` enqueues 1,000 elements in `0.0437 ms` and dequeues in `0.029 ms`.

2. When it comes to concatenation, **`Linked Lists`** (`DLL`/`SLL`) are much better.

   `Linked Lists` (`SLL`/`DLL`) excel in concatenation because they can simply link two lists together in `O(1)` time, whereas the `Array Queue` needs to create a new array and copy all elements into it.

3. If you want a balance between memory and performance, choose **`SLL`**.ions.

---

## Recommendations

### Use **Array-based Queue**:
- When basic operations (`enqueue`, `dequeue`, `peek`) are the primary focus.
- When memory efficiency is crucial.
- When concatenation operations are rare.

### Use **DLL Queue**:
- When frequent concatenation is required.
- When bidirectional traversal is needed.
- When dynamic size changes are common.

### Use **SLL Queue**:
- When memory efficiency is important.
- When unidirectional traversal suffices.
- When concatenation operations are frequent.

---

# Conclusion

The choice of queue implementation depends on the specific requirements of the application:
- **Array-based Queue** is ideal for basic operations and memory efficiency.
- **DLL** is suitable for applications requiring flexibility and frequent concatenation.
- **SLL** strikes a balance between memory efficiency and functionality.

---

# Future Work

- Analyze performance under varying workloads and larger data sizes.
- Measure memory usage across different implementations.
- Explore hybrid implementations that combine the strengths of different approaches.